In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

import colorsys
import os
from timeit import default_timer as timer

import numpy as np
from keras import backend as K
from keras.models import load_model
from keras.layers import Input
from PIL import Image, ImageFont, ImageDraw

from yolo3.model import yolo_eval, yolo_body, tiny_yolo_body
from yolo3.utils import letterbox_image
import os
from keras.utils import multi_gpu_model

Using TensorFlow backend.


In [2]:
import random

In [3]:
class YOLO(object):
    _defaults = {
        "model_path": 'model_data/yolo-tiny.h5',
        "anchors_path": 'model_data/tiny_yolo_anchors.txt',
        "classes_path": 'model_data/coco_classes.txt',
        "score" : 0.3,
        "iou" : 0.45,
        "model_image_size" : (416, 416),
        "gpu_num" : 1,
    }

    @classmethod
    def get_defaults(cls, n):
        if n in cls._defaults:
            return cls._defaults[n]
        else:
            return "Unrecognized attribute name '" + n + "'"

    def __init__(self, **kwargs):
        self.__dict__.update(self._defaults) # set up default values
        self.__dict__.update(kwargs) # and update with user overrides
        self.class_names = self._get_class()
        self.anchors = self._get_anchors()
        self.sess = K.get_session()
        self.boxes, self.scores, self.classes = self.generate()

    def _get_class(self):
        classes_path = os.path.expanduser(self.classes_path)
        with open(classes_path) as f:
            class_names = f.readlines()
        class_names = [c.strip() for c in class_names]
        return class_names

    def _get_anchors(self):
        anchors_path = os.path.expanduser(self.anchors_path)
        with open(anchors_path) as f:
            anchors = f.readline()
        anchors = [float(x) for x in anchors.split(',')]
        return np.array(anchors).reshape(-1, 2)

    def generate(self):
        model_path = os.path.expanduser(self.model_path)
        assert model_path.endswith('.h5'), 'Keras model or weights must be a .h5 file.'

        # Load model, or construct model and load weights.
        num_anchors = len(self.anchors)
        num_classes = len(self.class_names)
        is_tiny_version = num_anchors==6 # default setting
        try:
            self.yolo_model = load_model(model_path, compile=False)
        except:
            self.yolo_model = tiny_yolo_body(Input(shape=(None,None,3)), num_anchors//2, num_classes) \
                if is_tiny_version else yolo_body(Input(shape=(None,None,3)), num_anchors//3, num_classes)
            self.yolo_model.load_weights(self.model_path) # make sure model, anchors and classes match
        else:
            assert self.yolo_model.layers[-1].output_shape[-1] == \
                num_anchors/len(self.yolo_model.output) * (num_classes + 5), \
                'Mismatch between model and given anchor and class sizes'

        print('{} model, anchors, and classes loaded.'.format(model_path))

        # Generate colors for drawing bounding boxes.
        hsv_tuples = [(x / len(self.class_names), 1., 1.)
                      for x in range(len(self.class_names))]
        self.colors = list(map(lambda x: colorsys.hsv_to_rgb(*x), hsv_tuples))
        self.colors = list(
            map(lambda x: (int(x[0] * 255), int(x[1] * 255), int(x[2] * 255)),
                self.colors))
        np.random.seed(10101)  # Fixed seed for consistent colors across runs.
        np.random.shuffle(self.colors)  # Shuffle colors to decorrelate adjacent classes.
        np.random.seed(None)  # Reset seed to default.

        # Generate output tensor targets for filtered bounding boxes.
        self.input_image_shape = K.placeholder(shape=(2, ))
        if self.gpu_num>=2:
            self.yolo_model = multi_gpu_model(self.yolo_model, gpus=self.gpu_num)
        boxes, scores, classes = yolo_eval(self.yolo_model.output, self.anchors,
                len(self.class_names), self.input_image_shape,
                score_threshold=self.score, iou_threshold=self.iou)
        return boxes, scores, classes

    def detect_bbox(self,image):
        start = timer()

        if self.model_image_size != (None, None):
            assert self.model_image_size[0]%32 == 0, 'Multiples of 32 required'
            assert self.model_image_size[1]%32 == 0, 'Multiples of 32 required'
            boxed_image = letterbox_image(image, tuple(reversed(self.model_image_size)))
        else:
            new_image_size = (image.width - (image.width % 32),
                              image.height - (image.height % 32))
            boxed_image = letterbox_image(image, new_image_size)
        image_data = np.array(boxed_image, dtype='float32')

        print(image_data.shape)
        image_data /= 255.
        image_data = np.expand_dims(image_data, 0)  # Add batch dimension.
        out_boxes, out_scores, out_classes = self.sess.run(
            [self.boxes, self.scores, self.classes],
            feed_dict={
                self.yolo_model.input: image_data,
                self.input_image_shape: [image.size[1], image.size[0]],
                K.learning_phase(): 0
            })

        print('Found {} boxes for {}'.format(len(out_boxes), 'img'))

        font = ImageFont.truetype(font='font/FiraMono-Medium.otf',
                    size=np.floor(3e-2 * image.size[1] + 0.5).astype('int32'))
        thickness = (image.size[0] + image.size[1]) // 300
        
        ret_bbox=[]
        ret_score=[]
        ret_class_names=[]
        for i, c in reversed(list(enumerate(out_classes))):
            #predicted_class = self.class_names[c]
            box = out_boxes[i]
            score = out_scores[i]

            #label = '{} {:.2f}'.format(predicted_class, score)
            #draw = ImageDraw.Draw(image)
            #label_size = draw.textsize(label, font)

            top, left, bottom, right = box
            top = max(0, np.floor(top + 0.5).astype('int32'))
            left = max(0, np.floor(left + 0.5).astype('int32'))
            bottom = min(image.size[1], np.floor(bottom + 0.5).astype('int32'))
            right = min(image.size[0], np.floor(right + 0.5).astype('int32'))
            print(self.class_names[c], (left, top), (right, bottom))
            
            ret_bbox.append((top, left, bottom, right))
            ret_score.append(score.round(2))
            ret_class_names.append(self.class_names[c])
            

        end = timer()
        print(end - start)
        return ret_bbox,ret_score,ret_class_names
        
    def detect_image(self, image):
        start = timer()

        if self.model_image_size != (None, None):
            assert self.model_image_size[0]%32 == 0, 'Multiples of 32 required'
            assert self.model_image_size[1]%32 == 0, 'Multiples of 32 required'
            boxed_image = letterbox_image(image, tuple(reversed(self.model_image_size)))
        else:
            new_image_size = (image.width - (image.width % 32),
                              image.height - (image.height % 32))
            boxed_image = letterbox_image(image, new_image_size)
        image_data = np.array(boxed_image, dtype='float32')

        print(image_data.shape)
        image_data /= 255.
        image_data = np.expand_dims(image_data, 0)  # Add batch dimension.
        out_boxes, out_scores, out_classes = self.sess.run(
            [self.boxes, self.scores, self.classes],
            feed_dict={
                self.yolo_model.input: image_data,
                self.input_image_shape: [image.size[1], image.size[0]],
                K.learning_phase(): 0
            })

        print('Found {} boxes for {}'.format(len(out_boxes), 'img'))

        font = ImageFont.truetype(font='font/FiraMono-Medium.otf',
                    size=np.floor(3e-2 * image.size[1] + 0.5).astype('int32'))
        thickness = (image.size[0] + image.size[1]) // 300

        for i, c in reversed(list(enumerate(out_classes))):
            predicted_class = self.class_names[c]
            box = out_boxes[i]
            score = out_scores[i]

            label = '{} {:.2f}'.format(predicted_class, score)
            draw = ImageDraw.Draw(image)
            label_size = draw.textsize(label, font)

            top, left, bottom, right = box
            top = max(0, np.floor(top + 0.5).astype('int32'))
            left = max(0, np.floor(left + 0.5).astype('int32'))
            bottom = min(image.size[1], np.floor(bottom + 0.5).astype('int32'))
            right = min(image.size[0], np.floor(right + 0.5).astype('int32'))
            print(label, (left, top), (right, bottom))

            if top - label_size[1] >= 0:
                text_origin = np.array([left, top - label_size[1]])
            else:
                text_origin = np.array([left, top + 1])

            # My kingdom for a good redistributable image drawing library.
            for i in range(thickness):
                draw.rectangle(
                    [left + i, top + i, right - i, bottom - i],
                    outline=self.colors[c])
            draw.rectangle(
                [tuple(text_origin), tuple(text_origin + label_size)],
                fill=self.colors[c])
            draw.text(text_origin, label, fill=(0, 0, 0), font=font)
            del draw

        end = timer()
        print(end - start)
        return image

    def close_session(self):
        self.sess.close()


In [4]:
def detect_video(yolo, video_path, output_path=""):
    import cv2
    vid = cv2.VideoCapture(video_path)
    if not vid.isOpened():
        raise IOError("Couldn't open webcam or video")
    video_FourCC    = int(vid.get(cv2.CAP_PROP_FOURCC))
    video_fps       = vid.get(cv2.CAP_PROP_FPS)
    video_size      = (int(vid.get(cv2.CAP_PROP_FRAME_WIDTH)),
                        int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    isOutput = True if output_path != "" else False
    if isOutput:
        print("!!! TYPE:", type(output_path), type(video_FourCC), type(video_fps), type(video_size))
        out = cv2.VideoWriter(output_path, video_FourCC, video_fps, video_size)
    accum_time = 0
    curr_fps = 0
    fps = "FPS: ??"
    prev_time = timer()
    while True:
        return_value, frame = vid.read()
        if return_value==False:
            break
        image = Image.fromarray(frame)
        image = yolo.detect_image(image)
        result = np.asarray(image)
        curr_time = timer()
        exec_time = curr_time - prev_time
        prev_time = curr_time
        accum_time = accum_time + exec_time
        curr_fps = curr_fps + 1
        if accum_time > 1:
            accum_time = accum_time - 1
            fps = "FPS: " + str(curr_fps)
            curr_fps = 0
        cv2.putText(result, text=fps, org=(3, 15), fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=0.50, color=(255, 0, 0), thickness=2)
        cv2.namedWindow("result", cv2.WINDOW_NORMAL)
        cv2.imshow("result", result)
        if isOutput:
            out.write(result)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    yolo.close_session()
    
def detect_img(yolo):
    while True:
        img = input('Input image filename:')
        try:
            image = Image.open(img)
        except:
            print('Open Error! Try again!')
            continue
        else:
            r_image = yolo.detect_image(image)
            print(type(r_image))
            import cv2
            cv2.imwrite("out.jpg", np.asarray(r_image)[..., ::-1])
            r_image.show()
    yolo.close_session()



In [5]:
#video_path='Cars Driving On Street3.mp4'
video_path='Pexels Videos 4788.mp4'

output_path='out.mp4'

In [6]:
yolo=YOLO()

model_data/yolo-tiny.h5 model, anchors, and classes loaded.


In [7]:
def bboxYOLO2CV2(bbox):
    ret_bbox=[]
    for box in bbox:
        ret_bbox.append((box[1],box[0],box[3]-box[1],box[2]-box[0]))
    return ret_bbox

In [8]:
class TrackingMixer():

    def __init__(self, video_size,first_frame):
        self.boxes = [(0,0,0,0)]
        self.boxstates=[1]
        self.multiTracker=cv2.MultiTracker_create()
        self.multiTracker.add(cv2.TrackerKCF_create(), first_frame, self.boxes[0])
        self.overlap_torelance=0.35
        self.tracking_stay_fps=10
        self.video_size=video_size
        self.first=True

    def MixBbox(self,detected_bbox,frame):

        self.UpdateBox(detected_bbox,frame)
        self.TrackerInitialize()
        
        ret_boxes=[self.boxes[i] for i in range(len(self.boxstates)) if self.boxstates[i]>1]
        
        return ret_boxes
                
    def TrackerInitialize(self):
        self.multiTracker=cv2.MultiTracker_create()
        for bbox in self.boxes:
            self.multiTracker.add(cv2.TrackerKCF_create(), frame, bbox)

        
    def UpdateBox(self,detected_bbox,frame):
        success, boxes = self.multiTracker.update(frame)
        if len(self.boxes)!=len(boxes):
            print('different')
        corresponding_boxes,new_boxes =self.RetCorrespondingBoxes(boxes,detected_bbox)
        
        ret_boxes=[]
        for i,box in enumerate(boxes):
            if corresponding_boxes[i]!=None:
                self.boxstates[i]=min(self.boxstates[i]+1,self.tracking_stay_fps)
                r=0.6
                ret_boxes.append(tuple(((np.array(box)*r+np.array(corresponding_boxes[i])*(1-r))).astype(np.int)))
            else:
                ret_boxes.append(tuple(box))
                self.boxstates[i]=max(self.boxstates[i]-1,0)
                
        ret_boxes=ret_boxes+new_boxes
        self.boxstates=self.boxstates+([1]*len(new_boxes))
        self.boxes=ret_boxes
        print('box',self.boxes)
        print('states',self.boxstates)
        self.RemoveZeroStateBox()
        self.RemoveBoxOnEdge()
        self.RemoveOverlapBox()
        
    def RemoveOverlapBox(self):
        i=0
        while True:
            for k in list(range(i+1,len(self.boxes)))[::-1]:
                if self.IsBboxOverlap(self.boxes[i],self.boxes[k],0.2):
                    del self.boxes[k]
                    del self.boxstates[k]
                    print('DELETE')
            i+=1
        
            if i>len(self.boxes)-3:
                break

        
    def RemoveZeroStateBox(self):
        self.boxes=[self.boxes[i] for i in range(len(self.boxes)) if self.boxstates[i]!=0]
        self.boxstates=[self.boxstates[i] for i in range(len(self.boxstates)) if self.boxstates[i]!=0]
        
        
    def RemoveBoxOnEdge(self):
        r=0.98
        for i,box in enumerate(self.boxes):
            left=box[0]
            right=box[0]+box[2]
            top=box[1]
            bottom=box[1]+box[3]
            if (left<self.video_size[0]*(1-r))|(right>self.video_size[0]*r)|(top<self.video_size[1]*(1-r))|(bottom>self.video_size[1]*r):
                self.boxes.remove(box)
                del self.boxstates[i]
        
        
    def ChooseBiggestBox(self,bbox_list):
        area=[]
        if len(bbox_list)>0:
            for bbox in bbox_list:
                area.append(bbox[2]*bbox[3])

            return bbox_list[np.array(area).argmax()]

        else:
            return None
        
    def RetCorrespondingBoxes(self,original_bbox,new_bbox):
        corresponding_boxes=[]
        new_boxes=new_bbox.copy()
        
        for o_bbox in original_bbox:
            temp_box=[]
            for n_bbox in new_boxes:
                if self.IsBboxOverlap(o_bbox,n_bbox,self.overlap_torelance):
                    temp_box.append(n_bbox)
                    new_boxes.remove(n_bbox)
            corresponding_boxes.append(self.ChooseBiggestBox(temp_box))
        return corresponding_boxes,new_boxes    
                    
    def IsBboxOverlap(self,bbox1,bbox2,ratio):
        center_x1=bbox1[0]+bbox1[2]/2
        center_y1=bbox1[1]+bbox1[3]/2
        center_x2=bbox2[0]+bbox2[2]/2
        center_y2=bbox2[1]+bbox2[3]/2
        
        
        if (np.abs(center_x1-center_x2)<((bbox1[2]+bbox2[2])/2*ratio))&(np.abs(center_y1-center_y2)<((bbox1[3]+bbox2[3])/2*ratio)):
            return True
        else:
            return False
        
            
        

In [9]:
vid = cv2.VideoCapture(video_path)


if not vid.isOpened():
    raise IOError("Couldn't open webcam or video")
video_FourCC    = int(vid.get(cv2.CAP_PROP_FOURCC))
video_fps       = vid.get(cv2.CAP_PROP_FPS)
video_size      = (int(vid.get(cv2.CAP_PROP_FRAME_WIDTH)),
                    int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT)))
isOutput = True if output_path != "" else False

 
# Read first frame
success, frame = vid.read()
# quit if unable to read the video file
if not success:
    print('Failed to read video')
    sys.exit(1)
    
## Select boxes
bboxes = []
colors = [] 

'''
# OpenCV's selectROI function doesn't work for selecting multiple objects in Python
# So we will call this function in a loop till we are done selecting all objects
while True:
      # draw bounding boxes over objects

        # selectROI's default behaviour is to draw box starting from the center
    # when fromCenter is set to false, you can draw box starting from top left corner
    bbox = cv2.selectROI('MultiTracker', frame)
    bboxes.append(bbox)
    colors.append((random.randint(0, 255), random.randint(0, 255), random.randint(0, 255)))
    print("Press q to quit selecting boxes and start tracking")
    print("Press any other key to select next object")
    k = cv2.waitKey(0) & 0xFF
    if (k == 113):  # q is pressed
        break
'''
print('Selected bounding boxes {}'.format(bboxes))
 
# Create MultiTracker object
#multiTracker = cv2.MultiTracker_create()
# Initialize MultiTracker 
#for bbox in bboxes:
#    multiTracker.add(cv2.TrackerKCF_create(), frame, bbox)


if isOutput:
    print("!!! TYPE:", type(output_path), type(video_FourCC), type(video_fps), type(video_size))
    out = cv2.VideoWriter(output_path, video_FourCC, video_fps, video_size)
accum_time = 0
curr_fps = 0
fps = "FPS: ??"
prev_time = cv2.getTickCount()

tm=TrackingMixer(video_size=video_size,first_frame=frame)

while True:
    return_value, frame = vid.read()
    if return_value==False:
        break
        
    # Start timer
    start = cv2.getTickCount()

    # トラッカーをアップデートする
    #success, boxes = multiTracker.update(frame)
    image = Image.fromarray(frame)
    pred_bbox,pred_score,pred_class_names=yolo.detect_bbox(image) #bbox 
    print('yolo finish')
    p_bbox=bboxYOLO2CV2(pred_bbox)
    #print('pred_bbox',p_bbox)
    # draw tracked objects
    #for i, newbox in enumerate(boxes):
    #    p1 = (int(newbox[0]), int(newbox[1]))
    #    p2 = (int(newbox[0] + newbox[2]), int(newbox[1] + newbox[3]))
    #    cv2.rectangle(frame, p1, p2, colors[i], 2, 1)
    # draw pred object
    #'''
    for i, newbox in enumerate(p_bbox):
        p1 = (int(newbox[0]), int(newbox[1]))
        p2 = (int(newbox[0] + newbox[2]), int(newbox[1] + newbox[3]))
        cv2.rectangle(frame, p1, p2, (0,255,0), 2, 2)
    #''' 
    '''
    adj_p_bbox=tm.MixBbox(p_bbox,frame)
    #print('adj_p_bbox',adj_p_bbox)
    # draw pred object
    for i, newbox in enumerate(adj_p_bbox):
    #for i, newbox in enumerate(tm.boxes):
        p1 = (int(newbox[0]), int(newbox[1]))
        p2 = (int(newbox[0] + newbox[2]), int(newbox[1] + newbox[3]))
        cv2.rectangle(frame, p1, p2, (0,0,255), 2, 1)
        
    #'''
        

    # FPSを計算する
    fps = cv2.getTickFrequency() / (cv2.getTickCount() - start);

    # 検出した場所に四角を書く
    '''
    if track:
        # Tracking success
        p1 = (int(bbox[0]), int(bbox[1]))
        p2 = (int(bbox[0] + bbox[2]), int(bbox[1] + bbox[3]))
        cv2.rectangle(frame, p1, p2, (0,255,0), 2, 1)
    else :
        # トラッキングが外れたら警告を表示する
        cv2.putText(frame, "Failure", (10,50), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1, cv2.LINE_AA);
    '''
    # FPSを表示する
    cv2.putText(frame, "FPS : " + str(int(fps)), (10,20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1, cv2.LINE_AA);

    image = Image.fromarray(frame)

    result = np.asarray(image)
    curr_time = cv2.getTickCount()
    exec_time = curr_time - prev_time
    prev_time = curr_time
    accum_time = accum_time + exec_time
    curr_fps = curr_fps + 1
    if accum_time > 1:
        accum_time = accum_time - 1
        fps = "FPS: " + str(curr_fps)
        curr_fps = 0
    cv2.putText(result, text=fps, org=(3, 15), fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                fontScale=0.50, color=(255, 0, 0), thickness=2)
    #cv2.namedWindow("result", cv2.WINDOW_NORMAL)
    #cv2.imshow("result", result)
    if isOutput:
        out.write(result)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    cv2.imshow('MultiTracker', frame)

# キャプチャをリリースして、ウィンドウをすべて閉じる
vid.release()
out.release()
cv2.destroyAllWindows()

Selected bounding boxes []
!!! TYPE: <class 'str'> <class 'int'> <class 'float'> <class 'tuple'>
(416, 416, 3)
Found 13 boxes for img
bus (985, 519) (1118, 627)
bus (1014, 694) (1323, 1014)
car (1449, 767) (1585, 844)
car (646, 608) (735, 731)
car (1637, 992) (1822, 1080)
car (1412, 984) (1570, 1079)
car (730, 534) (840, 629)
car (670, 915) (870, 1077)
car (985, 519) (1118, 627)
car (864, 543) (973, 637)
car (835, 983) (1015, 1080)
car (1144, 1003) (1325, 1080)
car (548, 764) (655, 870)
2.5725327
yolo finish
(416, 416, 3)
Found 12 boxes for img
bus (986, 522) (1120, 628)
bus (1016, 699) (1319, 1005)
car (647, 603) (736, 733)
car (1642, 995) (1819, 1080)
car (867, 543) (973, 636)
car (1405, 984) (1570, 1073)
car (731, 536) (842, 630)
car (986, 522) (1120, 628)
car (671, 914) (864, 1077)
car (837, 986) (1018, 1080)
car (1147, 1004) (1320, 1080)
car (546, 765) (655, 868)
0.05082969999999998
yolo finish
(416, 416, 3)
Found 13 boxes for img
bus (988, 524) (1121, 632)
bus (1022, 671) (1376, 

(416, 416, 3)
Found 12 boxes for img
bus (1191, 625) (1389, 828)
bus (785, 457) (902, 569)
bus (995, 707) (1288, 1013)
car (1827, 666) (1917, 786)
car (1371, 986) (1528, 1080)
car (639, 678) (745, 761)
car (1132, 996) (1288, 1080)
car (546, 772) (668, 878)
car (651, 935) (876, 1080)
car (1439, 836) (1569, 939)
car (827, 1001) (998, 1080)
car (433, 899) (574, 1007)
0.0525171999999996
yolo finish
(416, 416, 3)
Found 13 boxes for img
bus (783, 456) (906, 567)
bus (996, 713) (1279, 1010)
car (713, 736) (880, 858)
car (1375, 987) (1522, 1080)
car (550, 770) (667, 878)
car (1825, 668) (1915, 787)
car (1597, 978) (1770, 1078)
car (651, 937) (875, 1080)
car (638, 681) (742, 766)
car (1437, 832) (1569, 941)
car (1132, 996) (1286, 1080)
car (826, 1000) (997, 1080)
car (432, 902) (574, 1008)
0.04716400000000043
yolo finish
(416, 416, 3)
Found 12 boxes for img
bus (778, 457) (904, 566)
bus (995, 716) (1278, 1009)
car (1592, 979) (1774, 1075)
car (1381, 985) (1524, 1080)
car (1275, 883) (1405, 983)

Found 9 boxes for img
bus (1015, 739) (1244, 999)
car (1337, 970) (1492, 1078)
car (789, 667) (901, 762)
car (953, 607) (1067, 711)
car (1117, 996) (1281, 1080)
car (1429, 819) (1558, 923)
car (1549, 966) (1730, 1077)
car (426, 906) (569, 1017)
car (541, 809) (657, 908)
0.05163979999999935
yolo finish
(416, 416, 3)
Found 8 boxes for img
bus (1016, 741) (1242, 1002)
car (786, 667) (900, 761)
car (952, 606) (1068, 711)
car (1116, 995) (1277, 1080)
car (1430, 820) (1559, 922)
car (1550, 966) (1725, 1079)
car (426, 906) (569, 1017)
car (539, 810) (656, 909)
0.05481149999999957
yolo finish
(416, 416, 3)
Found 8 boxes for img
bus (1015, 742) (1242, 1000)
car (785, 667) (900, 762)
car (950, 606) (1065, 710)
car (1115, 994) (1276, 1080)
car (1550, 965) (1719, 1079)
car (1428, 823) (1559, 924)
car (426, 907) (568, 1018)
car (538, 808) (656, 909)
0.045470499999999525
yolo finish
(416, 416, 3)
Found 8 boxes for img
bus (1010, 740) (1243, 1000)
car (785, 666) (899, 760)
car (1113, 993) (1277, 1080

(416, 416, 3)
Found 11 boxes for img
bus (1017, 760) (1224, 986)
car (718, 857) (961, 1052)
car (1526, 952) (1670, 1061)
car (1319, 946) (1466, 1070)
car (1647, 991) (1817, 1078)
car (862, 766) (975, 875)
car (792, 671) (896, 762)
car (426, 906) (562, 1016)
car (1100, 990) (1274, 1080)
car (1394, 825) (1532, 930)
car (539, 813) (655, 919)
0.05193109999999734
yolo finish
(416, 416, 3)
Found 9 boxes for img
bus (1016, 763) (1224, 985)
car (1320, 942) (1463, 1069)
car (790, 671) (896, 761)
car (1654, 994) (1816, 1080)
car (862, 768) (978, 873)
car (1102, 988) (1273, 1080)
car (426, 905) (565, 1017)
car (1393, 820) (1531, 926)
car (540, 812) (654, 917)
0.04730460000000036
yolo finish
(416, 416, 3)
Found 9 boxes for img
bus (1016, 763) (1223, 984)
car (789, 672) (894, 762)
car (1103, 986) (1273, 1080)
car (1318, 943) (1465, 1068)
car (863, 768) (979, 872)
car (1650, 989) (1816, 1080)
car (428, 906) (566, 1017)
car (1394, 819) (1528, 921)
car (540, 813) (654, 916)
0.06317149999999927
yolo fi

(416, 416, 3)
Found 11 boxes for img
truck (994, 683) (1196, 930)
bus (994, 683) (1196, 930)
car (663, 932) (863, 1080)
car (623, 694) (751, 783)
car (1307, 867) (1477, 1047)
car (1363, 798) (1486, 907)
car (545, 798) (662, 895)
car (1219, 815) (1325, 938)
car (855, 749) (970, 858)
car (1580, 966) (1777, 1073)
car (426, 898) (575, 1013)
0.06342319999999901
yolo finish
(416, 416, 3)
Found 12 boxes for img
truck (993, 676) (1197, 936)
bus (993, 676) (1197, 936)
car (788, 606) (903, 713)
car (629, 694) (752, 783)
car (1223, 813) (1330, 939)
car (1312, 884) (1456, 1026)
car (659, 935) (867, 1080)
car (857, 747) (971, 856)
car (1382, 772) (1490, 870)
car (550, 796) (661, 893)
car (1576, 964) (1774, 1077)
car (426, 895) (578, 1013)
0.05822419999999795
yolo finish
(416, 416, 3)
Found 11 boxes for img
truck (993, 670) (1199, 938)
bus (993, 670) (1199, 938)
car (657, 932) (871, 1080)
car (632, 693) (753, 779)
car (1222, 811) (1331, 935)
car (857, 742) (972, 851)
car (1313, 885) (1457, 1033)
car

...

Found 14 boxes for img
car (1229, 681) (1323, 791)
car (1556, 662) (1656, 759)
car (1169, 880) (1332, 986)
car (1307, 765) (1425, 859)
car (694, 669) (809, 762)
car (1207, 833) (1331, 931)
car (1551, 879) (1710, 996)
car (483, 827) (613, 930)
car (1078, 667) (1185, 811)
car (1154, 992) (1327, 1080)
car (447, 924) (628, 1080)
car (1314, 800) (1439, 917)
car (582, 741) (710, 853)
car (882, 648) (975, 754)
0.06176169999999814
yolo finish
(416, 416, 3)
Found 14 boxes for img
car (1310, 763) (1427, 857)
car (1230, 680) (1322, 790)
car (899, 761) (1085, 968)
car (1205, 832) (1334, 933)
car (693, 668) (810, 760)
car (1560, 660) (1659, 757)
car (1552, 880) (1710, 994)
car (485, 829) (616, 932)
car (446, 923) (626, 1080)
car (1316, 802) (1440, 918)
car (1155, 992) (1328, 1080)
car (1077, 669) (1185, 811)
car (584, 741) (714, 851)
car (883, 649) (980, 753)
0.0527232999999967
yolo finish
(416, 416, 3)
Found 12 boxes for img
car (694, 667) (811, 760)
car (1310, 761) (1430, 857)
car (1537, 873) (17

Found 10 boxes for img
car (1614, 692) (1712, 783)
car (702, 666) (812, 760)
car (1200, 831) (1343, 946)
car (883, 643) (973, 752)
car (1397, 824) (1483, 941)
car (1079, 655) (1176, 793)
car (1543, 845) (1661, 949)
car (587, 741) (716, 854)
car (1297, 760) (1412, 866)
car (1143, 982) (1325, 1080)
0.06652859999999805
yolo finish
(416, 416, 3)
Found 9 boxes for img
car (702, 667) (812, 759)
car (884, 643) (975, 751)
car (1199, 833) (1342, 944)
car (1397, 826) (1483, 938)
car (1078, 655) (1175, 792)
car (1543, 844) (1661, 950)
car (588, 741) (715, 854)
car (1301, 760) (1410, 866)
car (1145, 980) (1326, 1080)
0.05245820000000023
yolo finish
(416, 416, 3)
Found 10 boxes for img
bus (728, 751) (912, 1009)
car (702, 668) (813, 761)
car (1397, 826) (1482, 937)
car (884, 642) (974, 752)
car (1078, 652) (1176, 791)
car (1540, 871) (1696, 960)
car (1198, 834) (1343, 943)
car (587, 740) (715, 854)
car (1304, 760) (1414, 864)
car (1145, 981) (1327, 1080)
0.04689300000000074
yolo finish
(416, 416, 3

(416, 416, 3)
Found 14 boxes for img
bus (1012, 595) (1148, 726)
bus (954, 471) (1070, 575)
bus (729, 745) (911, 1008)
car (1408, 794) (1481, 903)
car (890, 610) (988, 731)
car (1125, 958) (1315, 1080)
car (452, 873) (614, 1065)
car (584, 738) (721, 843)
car (1396, 1010) (1591, 1080)
car (1292, 757) (1411, 869)
car (1654, 980) (1847, 1076)
car (1198, 835) (1345, 942)
car (1697, 734) (1800, 817)
car (1541, 827) (1673, 932)
0.06453020000000009
yolo finish
(416, 416, 3)
Found 14 boxes for img
bus (1011, 594) (1147, 724)
bus (954, 470) (1069, 576)
bus (730, 746) (910, 1006)
car (1409, 792) (1481, 903)
car (889, 607) (987, 732)
car (453, 873) (615, 1065)
car (1125, 959) (1314, 1080)
car (586, 737) (721, 842)
car (1294, 757) (1414, 867)
car (1657, 974) (1850, 1077)
car (1400, 1008) (1586, 1080)
car (1700, 735) (1806, 819)
car (1200, 835) (1346, 941)
car (1539, 825) (1672, 934)
0.05079560000000072
yolo finish
(416, 416, 3)
Found 14 boxes for img
bus (1013, 594) (1148, 721)
bus (956, 468) (106

(416, 416, 3)
Found 13 boxes for img
bus (887, 590) (989, 702)
bus (951, 464) (1068, 568)
bus (979, 562) (1161, 756)
bus (726, 728) (913, 997)
car (948, 878) (1136, 1012)
car (1629, 959) (1832, 1067)
car (887, 590) (989, 702)
car (1116, 957) (1320, 1080)
car (617, 737) (728, 855)
car (1397, 989) (1573, 1073)
car (457, 873) (619, 1055)
car (1514, 802) (1653, 896)
car (1215, 807) (1361, 923)
0.06532510000000258
yolo finish
(416, 416, 3)
Found 12 boxes for img
bus (888, 592) (988, 703)
bus (952, 461) (1070, 569)
bus (978, 561) (1160, 755)
bus (725, 729) (914, 997)
car (1114, 936) (1308, 1074)
car (888, 592) (988, 703)
car (1631, 959) (1820, 1062)
car (1401, 989) (1570, 1076)
car (583, 733) (724, 843)
car (456, 872) (619, 1055)
car (1219, 806) (1362, 923)
car (1515, 798) (1652, 896)
0.04856410000000011
yolo finish
(416, 416, 3)
Found 11 boxes for img
bus (954, 461) (1071, 571)
bus (978, 560) (1158, 756)
bus (723, 722) (916, 998)
car (888, 592) (990, 702)
car (1629, 961) (1810, 1058)
car (5

(416, 416, 3)
Found 13 boxes for img
bus (983, 554) (1157, 755)
bus (735, 738) (911, 991)
car (497, 841) (630, 931)
car (1249, 762) (1380, 873)
car (1599, 897) (1774, 1008)
car (613, 726) (738, 847)
car (999, 676) (1129, 780)
car (464, 875) (622, 1053)
car (1124, 922) (1295, 1073)
car (1384, 960) (1522, 1080)
car (1240, 806) (1383, 902)
car (1493, 752) (1628, 872)
person (210, 941) (290, 1076)
0.04998319999999978
yolo finish
(416, 416, 3)
Found 12 boxes for img
bus (981, 553) (1154, 756)
bus (735, 738) (912, 991)
car (497, 840) (633, 930)
car (1592, 892) (1778, 1007)
car (1387, 682) (1479, 798)
car (996, 676) (1127, 782)
car (613, 724) (737, 846)
car (465, 875) (624, 1053)
car (1123, 919) (1295, 1073)
car (1241, 804) (1385, 902)
car (1382, 956) (1522, 1080)
car (1492, 749) (1631, 871)
0.05109260000000404
yolo finish
(416, 416, 3)
Found 12 boxes for img
bus (979, 552) (1151, 753)
bus (736, 740) (910, 992)
car (1386, 683) (1478, 800)
car (998, 677) (1125, 783)
car (1590, 894) (1781, 1012

In [ ]:
cv2.destroyAllWindows()


In [ ]:
out.release()

In [ ]:
tm.boxstates

In [ ]:
multiTracker = cv2.MultiTracker_create()
# Initialize MultiTracker 

for bbox in bboxes:
    bbox=()
    multiTracker.add(cv2.TrackerKCF_create(), frame, bbox)
